In [1]:
%matplotlib inline

import torch

import triton
import triton.language as tl

torch.cuda.set_device(0)

# Add Kernel

In [2]:
# `@triton.jit` 可以理解为告诉编译器这是一个 Triton 的 kernel 函数
@triton.jit
def add_kernel(x_ptr,  # 指向向量 x 第一个元素的指针
               y_ptr,  # 指向向量 y 第一个元素的指针
               output_ptr,  
               n_elements,  # Size of the vector.
               BLOCK_SIZE: tl.constexpr,  # 可以认为是当前 kernel 的超参数
               ):
    # 当前 block 的 index，暂时跳过，后面会具体说明，可以暂时理解成我们平时写的 for 循环的索引（变量 i）
    pid = tl.program_id(axis=0)
    # 当前 block 第一个元素的索引
    block_start = pid * BLOCK_SIZE
    # 当前 block 中所有元素的索引
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    mask = offsets < n_elements
    # `tl.load` 会从 DRAM 中加载出向量 x、y 对应的 block 中的所有元素
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)
    # 真正的计算只有这一行
    output = x + y
    # 从 SM 中把计算得到的结果写回 DRAM
    tl.store(output_ptr + offsets, output, mask=mask)

# Kernel 的封装

In [3]:
def add(x: torch.Tensor, y: torch.Tensor):
    output = torch.empty_like(x)
    assert x.is_cuda and y.is_cuda and output.is_cuda
    n_elements = output.numel()

    # `triton.cdiv` 的功能是向上取整
    # `grid` 的功能是计算出当前的计算需要划分出多少个 block
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )

    # `add_kernel`` 的调用
    # `grid` 和 `add_kernel` 共享参数
    # meta 表示 `add_kernel`` 的参数，所以 meta['BLOCK_SIZE'] 相当于取出了 BLOCK_SIZE，这里就是512
    add_kernel[grid](x, y, output, n_elements, BLOCK_SIZE=512)

    return output

# 单元测试

让我们来测试一下我们的 kernel 是否正确 👀

In [4]:
torch.manual_seed(0)
size = 98432
x = torch.rand(size, device='cuda')
y = torch.rand(size, device='cuda')

# pytorch 实现
output_torch = x + y
# Triton 实现
output_triton = add(x, y)

# 结果打印
print(output_torch)
print(output_triton)
print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(output_torch - output_triton))}')

tensor([1.3713, 1.3076, 0.4940,  ..., 1.1147, 1.1906, 1.5746], device='cuda:0')
tensor([1.3713, 1.3076, 0.4940,  ..., 1.1147, 1.1906, 1.5746], device='cuda:0')
The maximum difference between torch and triton is 0.0
